- recursively gather all `.csv` files in your subdirectories
- concatenate them into one big dataframe

You can do this easily in Python with `pandas` and `glob`

In [9]:
import pandas as pd
import glob
import os

def collect_csvs(suffix="_defect_area.csv"):
    # Find all matching csv files
    csv_files = glob.glob(f"**/*{suffix}", recursive=True)

    # Exclude already combined files
    csv_files = [f for f in csv_files if not os.path.basename(f).startswith("combined")]
    
    dfs = []
    for file in csv_files:
        df = pd.read_csv(file)
        dfs.append(df)

    combined_df = pd.concat(dfs, ignore_index=True)

    # Show preview
    print(combined_df.head())
    print(combined_df.tail())
    
    # Output filename derived from suffix
    output_file = f"combined_{suffix}"
    combined_df.to_csv(output_file, index=False)

    print(f"Collected {len(csv_files)} files into {output_file}")

In [10]:
# Example usage:
suffix = "_defect_area.csv"

collect_csvs(suffix)

      sample  defect_area_mm2
0  Gomez_2p3        10.507689
1  Gomez_2p4         5.012462
2  Gomez_4p6         7.903992
3  Gomez_3p3         9.905482
4  Gomez_2p2        12.377603
       sample  defect_area_mm2
20  Gomez_3p1         8.459768
21  Gomez_3p6        11.463010
22  Gomez 5p5        12.406270
23  Gomez_1p3        11.815759
24  Gomez_4p3        10.306166
Collected 25 files into combined__defect_area.csv
